# RFM 분석

- Decil 분석의 단점을 보완한 분석 기법이 RFM 분석!

1. 구매 가능성이 높은 고객을 식별하기 위한 데이터 분석 방법

2. 마케팅에서 사용자 타겟팅을 위한 방법

-  지표 및 우선순위

1. Recency : 얼마나 최근에 구매?, 

2. Frequency : 얼마나 빈번하게 구매?

3. Monetary : 얼마나 많은 금액을 지불?



In [1]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")


In [2]:
# 데이터 불러오기
orders = pd.read_csv('./data/olist_orders_dataset.csv')
order_payments = pd.read_csv('./data/olist_order_payments_dataset.csv')
order_items = pd.read_csv('./data/olist_order_items_dataset.csv')
customers = pd.read_csv('./data/olist_customers_dataset.csv')
products = pd.read_csv('./data/olist_products_dataset.csv')
sellers = pd.read_csv('./data/olist_sellers_dataset.csv')
order_reviews = pd.read_csv('./data/olist_order_reviews_dataset.csv')
products_cat_translated= pd.read_csv('./data/product_category_name_translation.csv')
geolocation= pd.read_csv('./data/olist_geolocation_dataset.csv')

#### /payment_sequential/ : 한 주문에 대하여 지불 수단이 두 개 이상일 경우, 지불 수단의 순서 (payment_sequential이 1이 아닌 경우의 order_id로 데이터 확인)
#### /payment_installments/ : 할부 개월
#### /review_creation_date/ : 만족도 조사가 고객에게 전송된 날짜
#### /customer_id/ : 주문 키
#### /timestamp/ : 날짜 + 시간 (!=time)
#### /order_approved_at/ : 지불 승인 타임 스탬프
#### /order_delivered_carrier_date/ : 상품이 물류사에 전달된 타임 스탬프
#### /order_estimated_delivery_date/ : 고객에게 통보된 예상 배송일

## 데이터 로드 및 개별 데이터 이해

###  1. orders ¶
#### 데이터에서 확인한 정보
- 각 customer_id 당 중복되는 order_id는 없다

In [3]:
print(orders.shape)
orders.head()

(99441, 8)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [4]:
print(orders.order_id.nunique(), orders.customer_id.nunique(), len(orders))

99441 99441 99441


In [5]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [6]:
orders[orders['order_status']=='canceled'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 625 entries, 397 to 99347
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       625 non-null    object
 1   customer_id                    625 non-null    object
 2   order_status                   625 non-null    object
 3   order_purchase_timestamp       625 non-null    object
 4   order_approved_at              484 non-null    object
 5   order_delivered_carrier_date   75 non-null     object
 6   order_delivered_customer_date  6 non-null      object
 7   order_estimated_delivery_date  625 non-null    object
dtypes: object(8)
memory usage: 43.9+ KB


In [47]:
orders.order_status.value_counts()

delivered     96478
shipped        1107
invoiced        314
processing      301
created           5
approved          2
Name: order_status, dtype: int64

In [7]:
orders.order_status.value_counts()

delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64

- order_purchased_timestamp & order_estimated_delivery_date: 고객의 주문과 동시에 자동으로 생생됨.
주문상태(order_status)별 각 날짜 컬럼 내의 NaN값에 대한 요약

주문상태에 대한 설명은 다음과 같이 해석해볼 수 있음

- delivered: 배송 완료.
- shipped: 결제 완료 및 배송 시작
- invoiced / approved / processing: 결제 완료 및 배송 준비중
- unavailable: 주문상태가 unavailable인 경우에는 주문승인날짜(approved_at)컬럼에 값이 있다. 따라서 재고없음 등의 사유로 seller 측에서 취소한 경우가 이에 해당됨
- canceled: 이 경우는 approved_at, delivered_carrier_date,  delivered_customer_date에 해당하는 날짜에 값이 있다. 따라서 주문상태가 canceled인 경우는 고객이 취소한 경우임


 유효한 주문만을 분석하기 위해 주문상태가 unavailable, canceled인 것은 제외필요

In [8]:
orders = orders[~((orders["order_status"] == 'unavailable') | (orders["order_status"] == 'canceled'))]

In [9]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98207 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       98207 non-null  object
 1   customer_id                    98207 non-null  object
 2   order_status                   98207 non-null  object
 3   order_purchase_timestamp       98207 non-null  object
 4   order_approved_at              98188 non-null  object
 5   order_delivered_carrier_date   97583 non-null  object
 6   order_delivered_customer_date  96470 non-null  object
 7   order_estimated_delivery_date  98207 non-null  object
dtypes: object(8)
memory usage: 6.7+ MB


###  2. payments ¶

#### 데이터에서 확인한 정보

- order_id컬럼에서 4,446개의 중복 행이 있음

- order_id가 같으면 payment_sequential에서 1,2,3 등 연속숫자로 행방향으로 기재되어 있음

    예)order_id == 'e6a66a8350bb88497954d37688ab123e'
    
    8 rows for this 'order_id'⇒'payment_sequential' is ranged from 1 to 8 for it.


- payment_type에는 5개의 종류가 있음

    만약 payment_installments > 1  ⇒ credit_card만 입력됨
    
    만약 payment_type == credit_card  => 구매일자와 주문승인일자의 간격이 그리 크지 않음
    
    특이사항으로, 'boleto'라는 브라질의 특수한 결제 방식이 있음: 현금후불결제로, 발행된 청구서를 갖고 지정된 은행, 상점, 약국 등에서 결제하는 방식인데, 브라질의 낮은 은행계좌보유율(35%)로 인해 생긴 제도임. 우리나라의 지로 결제와 비슷하게 생각하면 됨.

In [10]:
print(order_payments.shape)
order_payments.head() 

(103886, 5)


,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [11]:
print(order_payments.order_id.nunique())

print("order_id의 값의 중복행 수 :", 103886 - 99440)

99440
order_id의 값의 중복행 수 : 4446


In [12]:
order_payments .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [13]:
# order_id컬럼에서 4,446개의 중복 행이 있음

tmp = order_payments[order_payments['order_id'].duplicated()]
tmp

,order_id,payment_sequential,payment_type,payment_installments,payment_value
1456,683bf306149bb869980b68d48a1bd6ab,1,credit_card,1,8.58
2324,e6a66a8350bb88497954d37688ab123e,2,voucher,1,10.51
2393,8e5148bee82a7e42c5f9ba76161dc51a,1,credit_card,1,0.67
2414,816ccd9d21435796e8ffa9802b2a782f,1,credit_card,1,5.65
2497,2cbcb371aee438c59b722a21d83597e0,2,voucher,1,7.80
...,...,...,...,...,...
103778,fd86c80924b4be8fb7f58c4ecc680dae,1,credit_card,1,76.10
103817,6d4616de4341417e17978fe57aec1c46,1,credit_card,1,19.18
103860,31bc09fdbd701a7a4f9b55b5955b8687,6,voucher,1,77.99
103869,c9b01bef18eb84888f0fd071b8413b38,1,credit_card,6,238.16


In [14]:
tmp[tmp.order_id == 'e6a66a8350bb88497954d37688ab123e']

,order_id,payment_sequential,payment_type,payment_installments,payment_value
2324,e6a66a8350bb88497954d37688ab123e,2,voucher,1,10.51
16289,e6a66a8350bb88497954d37688ab123e,6,voucher,1,17.30
39743,e6a66a8350bb88497954d37688ab123e,1,voucher,1,4.95
42000,e6a66a8350bb88497954d37688ab123e,3,voucher,1,5.75
45326,e6a66a8350bb88497954d37688ab123e,7,voucher,1,16.11
71825,e6a66a8350bb88497954d37688ab123e,5,voucher,1,19.16
96318,e6a66a8350bb88497954d37688ab123e,8,voucher,1,9.83


In [15]:
# payment_type에는 5개의 종류가 있음

order_payments['payment_type'].value_counts()

credit_card    76795
boleto         19784
voucher         5775
debit_card      1529
not_defined        3
Name: payment_type, dtype: int64

In [16]:
# payment_installments > 1 ⇒ credit_card만 입력됨

order_payments[order_payments['payment_installments'] > 1]['payment_type'].value_counts()

credit_card    51338
Name: payment_type, dtype: int64

In [17]:
order_payments[(order_payments['payment_type'] == 'credit_card') & (order_payments['payment_sequential'] > 1)]

,order_id,payment_sequential,payment_type,payment_installments,payment_value
199,332c6742772f2df936696b6512b10edb,2,credit_card,6,134.55
949,98f0e2b0d85754ca3b4e06d70f0f63a2,2,credit_card,2,70.00
1102,5373c26702420c0f842871e489365e20,2,credit_card,1,12.59
1135,eca3005a4f60ceb77b67a30b74095ddf,2,credit_card,1,3.99
1294,b18478bbf9b2283e3d12452f69fd9ce2,2,credit_card,1,79.92
...,...,...,...,...,...
102295,c75e9a685439f75e990bcf13443f02c5,2,credit_card,4,497.54
102341,407cc9b2c7575dbff71cb23b6cc26814,2,credit_card,7,76.39
102674,ef13a698cf88c9e17ccca0261e0b9e7a,2,credit_card,8,94.49
102724,55adc5cef9556477770f0b858669f8fc,2,credit_card,2,65.55


In [18]:
order_payments[order_payments['payment_type'] == 'not_defined']

,order_id,payment_sequential,payment_type,payment_installments,payment_value
51280,4637ca194b6387e2d538dc89b124b0ee,1,not_defined,1,0.0
57411,00b1cb0320190ca0daa2c88b35206009,1,not_defined,1,0.0
94427,c8c528189310eaa44a745b8d9d26908b,1,not_defined,1,0.0


### 3. order_reviews ¶
#### 데이터에서 확인한 정보

- review_id와 order_id 컬럼에서 중복된 행들이 존재함

    - 만약 여러 행의 review_id가 동일하다면, order_id를 제외한 나머지가 전부 동일함.
    - 반대로, 여러 행의 order_id가 동일하다면, 모든 컬럼의 값이 다 다름
    - 데이터를 통해서는 review_id와 order_id간의 유의미한 관계를 찾기 어려울 것으로 판단하였음
    - 적어도 order_id를 이용하여 상품, 판매자별 review_score를 분석해보는 것은 할 수 있을 것으로 보임

- review_score : 4, 5가 77% 정도로 대부분을 차지함

- review_comment_title, review_comment_message에 각각 88.3%, 58.7% 가 결측치임



In [19]:
print(order_reviews.shape)
order_reviews.head()

(99224, 7)


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [20]:
print(order_reviews['review_id'].nunique())
print(order_reviews['order_id'].nunique())

98410
98673


In [21]:
# review_id 중복값
tmp = order_reviews[order_reviews['review_id'].duplicated()]
print(tmp.shape)
tmp.head()

(814, 7)


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
3317,3242cc306a9218d0377831e175d62fbf,9c5bfba7de6a4abbb6ba0baab78d1622,5,NaN,Ótimo,2017-07-21 00:00:00,2017-07-22 01:06:37
5719,308316408775d1600dad81bd3184556d,3fe4dbcdb046a475dbf25463c1ca78bd,5,NaN,Ajudem a rastrear uma das 3 prateleiras que fa...,2017-09-07 00:00:00,2017-09-11 09:58:09
7213,8ee90ac383cf825bb7f4756130d4e74a,75d5d3d16567a27eefc5752aeb063072,5,NaN,recomendo,2017-05-30 00:00:00,2017-06-02 18:16:17
7805,d433c252647c51309432ca0b763f969b,191abde9c3bf77d1ce67d100d8db6d15,1,NaN,NaN,2017-09-03 00:00:00,2017-09-06 16:11:00
9013,03a6a25db577d0689440933055111897,3fde8b7313af6b37b84b5c7594d7add0,5,NaN,Muito Bom! Gostei Bastante! Tecido Ótimo! Aten...,2017-12-15 00:00:00,2017-12-16 01:32:18


In [22]:
# 'review_id' 중복값 예시 

order_reviews[order_reviews['review_id'] == '308316408775d1600dad81bd3184556d']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
4545,308316408775d1600dad81bd3184556d,27e5ed2ecefbe0bcc9ae0c9b26b763c3,5,NaN,Ajudem a rastrear uma das 3 prateleiras que fa...,2017-09-07 00:00:00,2017-09-11 09:58:09
5719,308316408775d1600dad81bd3184556d,3fe4dbcdb046a475dbf25463c1ca78bd,5,NaN,Ajudem a rastrear uma das 3 prateleiras que fa...,2017-09-07 00:00:00,2017-09-11 09:58:09
53661,308316408775d1600dad81bd3184556d,8b3c2785144e72ccba9b0213f0f1cd1e,5,NaN,Ajudem a rastrear uma das 3 prateleiras que fa...,2017-09-07 00:00:00,2017-09-11 09:58:09


In [23]:
# 'order_id' 중복값

tmp = order_reviews[order_reviews['order_id'].duplicated()]
print(tmp.shape)
tmp.head()

(551, 7)


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
1119,46abf3ea0b2710ad41390fdb79c32d84,5040757d4e06a4be96d3827b860b4e7c,5,NaN,NaN,2017-11-07 00:00:00,2017-11-10 20:07:48
3109,aa193e76d35950c4ae988237bb36ed2b,cf73e2cb1f4a9480ed70c154da3d954a,5,NaN,NaN,2018-01-18 00:00:00,2018-01-18 17:36:45
8108,40294ea5a778dc62080d6b3f55d361ce,e1bc1083cd7acd30d0576335373b907d,5,NaN,NaN,2018-03-23 00:00:00,2018-03-24 00:23:06
9064,32e2c7e889f7a185d462265398ee3631,c7cfea0c153e6382e32e84c2a9dd7d2e,5,NaN,NaN,2017-10-21 00:00:00,2017-10-22 00:47:25
9795,95a3135743556b117d888cc8c6e12e11,f9c78e6e58306dc81efbbada1ac11f24,3,NaN,O produto chegou antes do prazo mas recebi um ...,2017-09-14 00:00:00,2017-09-17 11:14:08


In [24]:
# 'order_id'중복값 예시 

order_reviews[order_reviews['order_id'] == 'f9c78e6e58306dc81efbbada1ac11f24']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
4753,3c95105686e2ae71e8fd94b6483eba79,f9c78e6e58306dc81efbbada1ac11f24,2,NaN,"comprei 4 cubas ao mesmo tempo, fui cobrada pe...",2017-09-19 00:00:00,2017-09-22 15:23:29
9795,95a3135743556b117d888cc8c6e12e11,f9c78e6e58306dc81efbbada1ac11f24,3,NaN,O produto chegou antes do prazo mas recebi um ...,2017-09-14 00:00:00,2017-09-17 11:14:08


In [25]:
# Summary of 'review_score' 
#전체값이 1인 상태에서 모든값을 비중으로 나누어 반환
order_reviews['review_score'].value_counts(normalize = True)

5    0.577763
4    0.192917
1    0.115133
3    0.082430
2    0.031756
Name: review_score, dtype: float64

In [26]:
# Proportion of null values

print(order_reviews['review_comment_title'].isnull().sum() / len(order_reviews))
print(order_reviews['review_comment_message'].isnull().sum() / len(order_reviews))

0.8834153027493349
0.5870253164556962


### 2-4. order_items ¶
#### 데이터에서 확인한 정보

- order_id, product_id, seller_id에 중복된 행들이 있음
    - 동일한 order_id는 같은 product_id, seller_id를 가짐
    - 동일한 order_id는 order_item_id컬럼에 1~N까지의 연속숫자를 가짐 ⇒해당 상품의 갯수에 따라 행이 늘어남
- order_payments데이터셋의 payment_value는 order_items데이터셋의 price + freight_value 임
    - 그러나 일치하지 않는 경우, 차액은 이자금액으로 판단
    

In [27]:
print(order_items.shape)
order_items.head()

(112650, 7)


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [28]:
# order_id 중복행 확인
tmp = order_items[order_items['order_id'].duplicated()]
print(tmp.shape)
tmp.head()

(13984, 7)


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
14,0008288aa423d2a3f00fcb17cd7d8719,2,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2018-02-21 02:55:52,49.90,13.37
33,00143d0f86d6fbd9f9b38ab440ac16f5,2,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.10
34,00143d0f86d6fbd9f9b38ab440ac16f5,3,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.10
43,001ab0a7578dd66cd4b0a71f5b6e1e41,2,0b0172eb0fd18479d29c3bc122c058c2,5656537e588803a555b8eb41f07a944b,2018-01-04 02:33:42,24.89,17.63
44,001ab0a7578dd66cd4b0a71f5b6e1e41,3,0b0172eb0fd18479d29c3bc122c058c2,5656537e588803a555b8eb41f07a944b,2018-01-04 02:33:42,24.89,17.63


In [29]:
# order_id 중복행 예시
order_items[order_items['order_id'] == '8272b63d03f5f79c56e9e4120aec44ef']

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
57297,8272b63d03f5f79c56e9e4120aec44ef,1,270516a3f41dc035aa87d220228f844c,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57298,8272b63d03f5f79c56e9e4120aec44ef,2,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57299,8272b63d03f5f79c56e9e4120aec44ef,3,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57300,8272b63d03f5f79c56e9e4120aec44ef,4,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57301,8272b63d03f5f79c56e9e4120aec44ef,5,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57302,8272b63d03f5f79c56e9e4120aec44ef,6,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57303,8272b63d03f5f79c56e9e4120aec44ef,7,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57304,8272b63d03f5f79c56e9e4120aec44ef,8,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57305,8272b63d03f5f79c56e9e4120aec44ef,9,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57306,8272b63d03f5f79c56e9e4120aec44ef,10,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89


In [30]:
order_items['order_item_id'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21])

In [31]:
#product_id 중복행 확인
tmp = order_items[order_items['product_id'].duplicated()]
print(tmp.shape)
tmp.head()

(79699, 7)


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
14,0008288aa423d2a3f00fcb17cd7d8719,2,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2018-02-21 02:55:52,49.90,13.37
33,00143d0f86d6fbd9f9b38ab440ac16f5,2,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.10
34,00143d0f86d6fbd9f9b38ab440ac16f5,3,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.10
41,0019c29108428acffd089c36103c9440,1,28b4eced95a52d9c437a4caf9d311b95,77530e9772f57a62c906e1c21538ab82,2018-03-12 06:50:26,59.90,19.95
43,001ab0a7578dd66cd4b0a71f5b6e1e41,2,0b0172eb0fd18479d29c3bc122c058c2,5656537e588803a555b8eb41f07a944b,2018-01-04 02:33:42,24.89,17.63


In [32]:
#seller_id 중복행 확인

tmp = order_items[order_items['seller_id'].duplicated()]
print(tmp.shape)
tmp.head()

(109555, 7)


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
14,0008288aa423d2a3f00fcb17cd7d8719,2,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2018-02-21 02:55:52,49.90,13.37
33,00143d0f86d6fbd9f9b38ab440ac16f5,2,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.10
34,00143d0f86d6fbd9f9b38ab440ac16f5,3,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.10
41,0019c29108428acffd089c36103c9440,1,28b4eced95a52d9c437a4caf9d311b95,77530e9772f57a62c906e1c21538ab82,2018-03-12 06:50:26,59.90,19.95
43,001ab0a7578dd66cd4b0a71f5b6e1e41,2,0b0172eb0fd18479d29c3bc122c058c2,5656537e588803a555b8eb41f07a944b,2018-01-04 02:33:42,24.89,17.63


### 1-5.products & product_cat_translation

#### 결과)

중복행은 없음
총 73개 카테고리, 32,951개의 제품이 있음
제품 수 top 1 카테고리: bed_bath_table(침대 욕실 테이블)

In [33]:
print(products.shape)
products.head()

(32951, 9)


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [34]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [35]:
print(products_cat_translated.shape)
products_cat_translated.head()

(71, 2)


,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [36]:
products_cat_translated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [37]:
#product_id 중복행 확인
products[products['product_id'].duplicated()]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm


In [38]:
print('The Number of Product Category: ', products['product_category_name'].nunique(), '\n')

print('Top 20 categoy by the number of product:')
products['product_category_name'].value_counts().head(20)

The Number of Product Category:  73 

Top 20 categoy by the number of product:


cama_mesa_banho                      3029
esporte_lazer                        2867
moveis_decoracao                     2657
beleza_saude                         2444
utilidades_domesticas                2335
automotivo                           1900
informatica_acessorios               1639
brinquedos                           1411
relogios_presentes                   1329
telefonia                            1134
bebes                                 919
perfumaria                            868
papelaria                             849
fashion_bolsas_e_acessorios           849
cool_stuff                            789
ferramentas_jardim                    753
pet_shop                              719
eletronicos                           517
construcao_ferramentas_construcao     400
eletrodomesticos                      370
Name: product_category_name, dtype: int64

### 6. sellers 
#### 데이터에서 확인한 정보
- 중복행 없음
- 총 3,095명의 판매자가 있음

In [39]:
print(sellers.shape)
sellers.head()

(3095, 4)


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [40]:
sellers[sellers['seller_id'].duplicated()]

,seller_id,seller_zip_code_prefix,seller_city,seller_state


### 7. customers ¶
#### 데이터에서 확인한 정보
- customer_id에는 중복 행 없으나, customer_unique_id에는 중복이 있음  ⇒ 단일고객은 결국 customer_unique_id의 96,096명임

In [41]:
print(customers.shape)
customers.head()

(99441, 5)


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [42]:
customers[customers['customer_id'].duplicated()]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state


In [43]:
tmp = customers[customers['customer_unique_id'].duplicated()]
print(tmp.shape)
tmp.head()

(3345, 5)


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
679,c57b4b6f3719475543b721e720a526ad,b6c083700ca8c135ba9f0f132930d4e8,8686,suzano,SP
1463,9f6f3da49e2d46e3a7529f5e3c25ecce,a40096fc0a3862e9e12bc55b5f8e6ab2,20561,rio de janeiro,RJ
1607,299f7b5125c8fbe1761a1b320c34fc7d,b8b3c435a58aebd788a477bed8342910,95585,arroio do sal,RS
2811,226d59f9f4b98e228b689eea45d03a6d,66980c3775537536f77b434d74e520f5,30290,belo horizonte,MG
3198,b1253701171dfb298f52a221f824e45b,788313d21c3507fe890921f6e17aa679,11070,santos,SP


In [44]:
print('The distinct number of customers:', format(customers['customer_unique_id'].nunique(),','), '\n')
customers[customers['customer_unique_id'] == 'a40096fc0a3862e9e12bc55b5f8e6ab2']

The distinct number of customers: 96,096 



,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
197,eee74b2a9aa0989243d2793c1a90fbf6,a40096fc0a3862e9e12bc55b5f8e6ab2,20561,rio de janeiro,RJ
1463,9f6f3da49e2d46e3a7529f5e3c25ecce,a40096fc0a3862e9e12bc55b5f8e6ab2,20561,rio de janeiro,RJ
35912,039bbed64cc5e2160b7ca4ac6b09449c,a40096fc0a3862e9e12bc55b5f8e6ab2,20561,rio de janeiro,RJ


### 8. geolocation 
#### 데이터에서 확인한 정보
위경도 정보 포함됨

In [45]:
print(geolocation.shape)
geolocation.head()

(1000163, 5)


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


---------------------------------

In [46]:
product = products['product_category_name'].tolist()
name = products_eng['product_category_name'].tolist()
eng_name = products_eng['product_category_name_english'].tolist()

for i in product:
    if i in name:
        product[product.index(i)] = eng_name[name.index(i)]

NameError: name 'products_eng' is not defined

In [ ]:
products['product_category_name'] = product

In [ ]:
products

In [ ]:
df = orders.merge(customers, on = 'customer_id', how = 'outer')
df = df.merge(items, on = 'order_id', how = 'outer')
df = df.merge(payments, on = 'order_id', how = 'outer')
df = df.merge(review, on = 'order_id', how = 'outer')
df = df.merge(products, on = 'product_id', how = 'outer')

In [ ]:
df

In [ ]:
df.info()

In [ ]:
print(df.isnull().sum(axis=0))